# GenoPhenoCorr Example 
## Phenotypic Traits associate with Variants on Gene MAPK8IP3


### TODO - Intro to package
BLAH BLAH BAahh

In [1]:
import sys
import glob
  
from genophenocorr import *
from genophenocorr import compare
from genophenocorr import compare_func as CF

## Importing Phenopacket Data

The phenopackets were created using [pyphetools](https://github.com/monarch-initiative/pyphetools) with data from [Platzer K., et al. (2019) De Novo Variants in MAPK8IP3 Cause Intellectual Disability with Variable Brain Anomalies](https://pubmed.ncbi.nlm.nih.gov/30612693/) and [Iwasawa S., et al. (2019) De Recurrent de novo MAPK8IP3 variants cause neurological phenotypes](https://pubmed.ncbi.nlm.nih.gov/30945334/). 


In [2]:
allPatients = AllPatients('../phenopackets/PlatzerData/*.json')

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


You can also add additional phenopackets to the AllPatients class by using AllPatients.add("Path/to/phenopacket.json"). 

In [3]:
##TO DO - use this to add the additional patient Ben(I think) found
# allPatients.add("json file to last subject")

## Examine the Data


In [4]:
allPatients.describe_all()

,Patient ID,Disease,Gene,Variant,Protein,HPO Terms
0,Individual 5,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001270, HP:0032988, HP:0025336, HP:003193..."
1,12,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001257, HP:0100021, HP:0010864]"
2,2,None,[MAPK8IP3],chr16 g.1706418G>T,ENSP00000501096,"[HP:0001251, HP:0010864]"
3,3,None,[MAPK8IP3],chr16 g.1706450C>G,ENSP00000501096,[HP:0002342]
4,13,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0002317, HP:0007301, HP:0002370, HP:0002342]"
5,Individual 4,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001270, HP:0032988, HP:0032989, HP:002533..."
6,4,None,[MAPK8IP3],chr16 g.1748705G>A,ENSP00000501096,[HP:0001256]
7,8,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001258, HP:0010864]"
8,Individual 3,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001270, HP:0025336, HP:0031936, HP:000218..."
9,Individual 2,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001270, HP:0032989, HP:0031936, HP:001086..."


We can also look more in depth as specific attributes of a patient. 

In [9]:
firstPat = allPatients.all_patients.get('1')
firstPat.describe()

ID                                                                   1
Disease ID                                                        None
Disease Label                                                     None
HPO IDs                                   ['HP:0001251', 'HP:0002342']
HPO Terms              ['Ataxia', 'Intellectual disability, moderate']
Variant                                    chr16 g.1706403_1706403delG
Gene Affected                                                 MAPK8IP3
Effect of Variant                                              p.G22fs
Protein Affected     C-Jun-amino-terminal kinase-interacting protein 3
Protein ID                                             ENSP00000501096
dtype: object

## Run Correlation Analysis
This function runs either a Chi^2^ test or a Fisher Exact test on the counts of each HPO Term that do or don't match a specific criteria. 
Format for function:
RunStats(AllPatients Class, Fun1, Fun2, extraVar_1, extraVar_2)

If Fun1 is CF.is_var_type, then extraVar_1 can be any variant type (missense, nonsense, deletion, etc.) and the function will count by HPO Term all the patients that have that type of variant and do or do not have that HPO Term. This will create the first half of the Contingency Table. Fun2 and extraVar_2 will do the same, creating the second half of the table. 

The example below creates a contingency table: <br>

| Patients with HPO Term and with missense variant | Patients without HPO Term and with missense variant | <br>
| Patients with HPO Term and with no missense variant | Patients without HPO Term and with no missense variant |


In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')


In [ ]:
for typ in allPatients.all_var_types:
    print(f'Comparing all in {typ} vs out of {typ}')
    compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, typ, typ)
    for typ2 in allPatients.all_var_types:
        if typ2 == typ:
            continue
        print(f'Comparing all in {typ} vs all in {typ2}')
        compare.RunStats(allPatients, CF.is_var_type, CF.is_var_type, typ, typ2)

In [ ]:
for var in allPatients.all_variants:
    print(f'Comparing affecting {var.variant_string} vs all not')
    compare.RunStats(allPatients, CF.is_var_match, CF.is_not_var_match, var, var)

In [ ]:
compare.RunStats(allPatients,CF.in_feature, CF.not_in_feature,'Domain','Domain')

In [ ]:
Protein.findProtein('ENSP00000501096')

In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')
